In [2]:

related = pd.read_csv('/Users/jingyichen/machinelearning/a-mimic/mimic-iii-clinical-database-1.4/CHARTEVENTS_Ventilation_Related_ITEMIDs.csv')
procedure = pd.read_csv('/Users/jingyichen/machinelearning/a-mimic/mimic-iii-clinical-database-1.4/PROCEDUREEVENTS_MV.csv')


In [1]:

import pandas as pd

charts = pd.read_csv('/Users/jingyichen/machinelearning/a-mimic/mimic-iii-clinical-database-1.4/CHARTEVENTS.csv', usecols=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID','CHARTTIME'])


In [3]:
import pandas as pd
import numpy as np

In [6]:
procedure.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258066 entries, 0 to 258065
Data columns (total 25 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   ROW_ID                      258066 non-null  int64  
 1   SUBJECT_ID                  258066 non-null  int64  
 2   HADM_ID                     258066 non-null  int64  
 3   ICUSTAY_ID                  257978 non-null  float64
 4   STARTTIME                   258066 non-null  object 
 5   ENDTIME                     258066 non-null  object 
 6   ITEMID                      258066 non-null  int64  
 7   VALUE                       258066 non-null  float64
 8   VALUEUOM                    113820 non-null  object 
 9   LOCATION                    52612 non-null   object 
 10  LOCATIONCATEGORY            52612 non-null   object 
 11  STORETIME                   258066 non-null  object 
 12  CGID                        258066 non-null  int64  
 13  ORDERID       

In [8]:
charts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330712483 entries, 0 to 330712482
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   SUBJECT_ID  int64  
 1   HADM_ID     int64  
 2   ICUSTAY_ID  float64
 3   ITEMID      int64  
 4   CHARTTIME   object 
dtypes: float64(1), int64(3), object(1)
memory usage: 12.3+ GB


In [4]:
icustays_df = pd.read_csv("/Users/jingyichen/machinelearning/a-mimic/mimic-iii-clinical-database-1.4/ICUSTAYS.csv")

In [10]:
icustays_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61532 entries, 0 to 61531
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ROW_ID          61532 non-null  int64  
 1   SUBJECT_ID      61532 non-null  int64  
 2   HADM_ID         61532 non-null  int64  
 3   ICUSTAY_ID      61532 non-null  int64  
 4   DBSOURCE        61532 non-null  object 
 5   FIRST_CAREUNIT  61532 non-null  object 
 6   LAST_CAREUNIT   61532 non-null  object 
 7   FIRST_WARDID    61532 non-null  int64  
 8   LAST_WARDID     61532 non-null  int64  
 9   INTIME          61532 non-null  object 
 10  OUTTIME         61522 non-null  object 
 11  LOS             61522 non-null  float64
dtypes: float64(1), int64(6), object(5)
memory usage: 5.6+ MB


In [5]:

# STEP 1: Extract relevant ITEMIDs for mechanical ventilation from CHARTEVENTS
vent_itemids_proc = [224385, 225475, 225792, 225794]
procedure_filtered = procedure[procedure['ITEMID'].isin(vent_itemids_proc)]

# Merge with icu time
proc_merged = procedure_filtered.merge(icustays_df[['ICUSTAY_ID', 'INTIME']], on='ICUSTAY_ID')
proc_merged['starttime'] = pd.to_datetime(proc_merged['STARTTIME'])
proc_merged['INTIME'] = pd.to_datetime(proc_merged['INTIME'])

# Filter for events within 24 hours of ICU admission
proc_vent = proc_merged[
    (proc_merged['starttime'] >= proc_merged['INTIME']) &
    (proc_merged['starttime'] <= proc_merged['INTIME'] + pd.Timedelta(hours=24))
]
proc_vent_flag = proc_vent[['ICUSTAY_ID']].drop_duplicates()
proc_vent_flag['mechanical_ventilation'] = 1



In [6]:
# STEP 2: Extract relevant ITEMIDs for mechanical ventilation from CHARTEVENTS
vent_itemids_chart = related['ITEMID'].unique().tolist()
charts_filtered = charts[charts['ITEMID'].isin(vent_itemids_chart)]

# Merge with icustays
charts_merged = charts_filtered.merge(icustays_df[['ICUSTAY_ID', 'INTIME']], on='ICUSTAY_ID')
charts_merged['CHARTTIME'] = pd.to_datetime(charts_merged['CHARTTIME'])
charts_merged['INTIME'] = pd.to_datetime(charts_merged['INTIME'])

# Filter for events within 24 hours of ICU admission
charts_vent = charts_merged[
    (charts_merged['CHARTTIME'] >= charts_merged['INTIME']) &
    (charts_merged['CHARTTIME'] <= charts_merged['INTIME'] + pd.Timedelta(hours=24))
]
charts_vent_flag = charts_vent[['ICUSTAY_ID']].drop_duplicates()
charts_vent_flag['mechanical_ventilation'] = 1



In [16]:
charts_vent_flag.info()
proc_vent_flag.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54875 entries, 0 to 16234650
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ICUSTAY_ID              54875 non-null  float64
 1   mechanical_ventilation  54875 non-null  int64  
dtypes: float64(1), int64(1)
memory usage: 1.3 MB
<class 'pandas.core.frame.DataFrame'>
Index: 8597 entries, 0 to 16733
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ICUSTAY_ID              8597 non-null   float64
 1   mechanical_ventilation  8597 non-null   int64  
dtypes: float64(1), int64(1)
memory usage: 201.5 KB


In [18]:
charts_vent_flag.head()

,ICUSTAY_ID,mechanical_ventilation
0,241249.0,1
5,290505.0,1
14,234044.0,1
240,291697.0,1
327,252542.0,1


In [15]:
# STEP 3: combine the flags
combined_flag = pd.concat([proc_vent_flag, charts_vent_flag]).drop_duplicates()

In [19]:
result = icustays_df[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID']].copy()

# Add mechanical_ventilation label
result = result.merge(combined_flag, on='ICUSTAY_ID', how='left')
result['mechanical_ventilation'] = result['mechanical_ventilation'].fillna(0).astype(int)

print(result.head())

   SUBJECT_ID  HADM_ID  ICUSTAY_ID  mechanical_ventilation
0         268   110404      280836                       1
1         269   106296      206613                       1
2         270   188028      220345                       1
3         271   173727      249196                       1
4         272   164716      210407                       1


In [20]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61532 entries, 0 to 61531
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   SUBJECT_ID              61532 non-null  int64
 1   HADM_ID                 61532 non-null  int64
 2   ICUSTAY_ID              61532 non-null  int64
 3   mechanical_ventilation  61532 non-null  int64
dtypes: int64(4)
memory usage: 1.9 MB


In [22]:
result.to_csv('/Users/jingyichen/machinelearning/a-mimic/mimic-iii-clinical-database-1.4/ventilation_flag.csv', index=True)